In [1]:
import os
import pandas as pd

os.environ["R_HOME"] = r"C:/Users/Jake/Documents/Anaconda3/envs/analysis_env/Lib/R"

In [2]:
import pymer4

In [3]:
from pymer4.utils import get_resource_path

df = pd.read_csv(os.path.join(get_resource_path(), "sample_data.csv"))

In [4]:

df.columns

Index(['Group', 'IV1', 'DV_l', 'DV', 'IV2', 'IV3'], dtype='object')

In [5]:
import matplotlib.pyplot as plt
from pymer4.models import Lmer

In [6]:
map_dict = {0.5: "sLTP", 1.0: "sLTD", 1.5: "Stable"}
#map_dict = {1: "sLTP", 0: "Stable"}
new_df = df.assign(Group_id=df["IV3"].map(map_dict))
#new_df = df.assign(Group_id=df["DV_l"].map(map_dict))
new_df

,Group,IV1,DV_l,DV,IV2,IV3,Group_id
0,1,20.0,0,7.936508,4.563492,0.5,sLTP
1,1,20.0,0,15.277778,0.000000,1.0,sLTD
2,1,20.0,1,0.000000,0.000000,1.5,Stable
3,1,20.0,1,9.523810,0.000000,0.5,sLTP
4,1,12.5,0,0.000000,0.000000,1.0,sLTD
...,...,...,...,...,...,...,...
559,53,8.0,0,48.412698,13.492063,1.0,sLTD
560,53,5.0,1,50.000000,70.039683,1.5,Stable
561,53,12.5,0,50.000000,69.047619,0.5,sLTP
562,53,5.0,0,50.000000,71.626984,1.0,sLTD


In [ ]:
import numpy as np
t=np.unique(new_df["DV_l"])

In [ ]:
type(t[0])

In [12]:
formula = "DV ~ DV_l + Group_id + DV_l:Group_id + (1|Group)"
#formula = "DV ~ DV_l*Group_id + (1|Group)"
#formula = "DV ~ Group_id + (1|Group)"
model = Lmer(formula, data=new_df)
model.fit(summarize=True, factors={"Group_id": ["sLTP", "sLTD", "Stable"], "DV_l":["0", "1"]}, ordered=True,
         rank=True, rank_group="Group_id")

KeyError: "['DV_l:Group_id'] not in index"

In [9]:
a = model.anova(force_orthogonal=True)

a

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Model refit with orthogonal polynomial contrasts)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
DV_l,17.687755,17.687755,1,513.939258,0.079238,0.778446,
Group_id,2485.315811,1242.657906,2,512.019567,5.566887,0.004057,**
DV_l:Group_id,440.688009,220.344005,2,514.176760,0.987102,0.373360,


In [ ]:
posthoc = model.post_hoc(marginal_vars=["DV_l"], grouping_vars=["Group_id"], p_adjust="fdr")

In [ ]:
posthoc[1]

In [ ]:
import numpy as np
from scipy import stats
sLTP_df = new_df[new_df["Group_id"]=="sLTP"]
sLTP_data = np.array(sLTP_df["DV"])
sLTD_df = new_df[new_df["Group_id"]=="sLTD"]
sLTD_data = np.array(sLTD_df["DV"])
stable_df = new_df[new_df["Group_id"]=="Stable"]
stable_data = np.array(stable_df["DV"])

In [ ]:
posthoc1 = model.post_hoc(marginal_vars=["DV_l"], grouping_vars=["Group_id"], p_adjust="fdr")
posthoc1[1]

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
t1, p1 = stats.ttest_ind(sLTP_data, sLTD_data)
t2, p2 = stats.ttest_ind(sLTP_data, stable_data)
t3, p3 = stats.ttest_ind(sLTD_data, stable_data)

In [ ]:
raw_pvals = [p1, p2, p3]
_, adj_pvals, _, _ = multipletests(
            raw_pvals,
            alpha=0.05,
            method="fdr_tsbh",
            is_sorted=False,
            returnsorted=False,
        )

In [ ]:
adj_pvals

In [ ]:
raw_pvals

In [ ]:
c = np.zeros(a.shape)
c[:, 2] = d
c

In [ ]:
data_dict = {"A": [1,1,1,1,1,1,1,np.nan], "B": [2,2,2,2,2,2,2,2], "C": [3,3,3,3,3,3,3,3]}
random_dict = {"A": ["a","a","a","b","b","b","c","c"], "B": ["a","a","a","b","b","b","c","c"], "C": ["a","a","a","b","b","b","c","c"]}

In [ ]:
df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data_dict.items()]))
new_df = df.dropna()
new_df

In [ ]:
df = pd.DataFrame.from_dict(data_dict)
df

In [ ]:
melted = pd.melt(df, value_vars=["A", "B", "C"], var_name="condition", value_name="data")
melted

In [ ]:
rand_df = pd.DataFrame.from_dict(random_dict)
rand_melt = pd.melt(rand_df, value_vars=random_dict.keys(), var_name="condition", value_name="rand_var")
rand_melt

In [ ]:
melted["rand_var"] = rand_melt["rand_var"]

In [ ]:
map_dict = {}
codes = list(range(len(data_dict.keys())))
for i, key in enumerate(data_dict.keys()):
    map_dict[key] = codes[i]
map_dict

In [ ]:
melted = melted.assign(condition_coded=melted["condition"].map(map_dict))

In [ ]:
melted

In [ ]:
import pingouin as pg
data_wide = pg.read_dataset('rm_anova_wide')
data_wide

In [ ]:
import numpy as np
dic = {"A": np.array([[1,2,3], [1,2,3]]), "B": np.array([[4,5,6], [4,5,6]]), "C": np.array([[7,8,9], [7,8,9]])}

#a = np.array(list(dic.values())).T
#a

In [ ]:
b = [a, a, a]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cross_decomposition import CCA
from sklearn.datasets import make_multilabel_classification
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

X, Y = make_multilabel_classification(
    n_classes=2, n_labels=1, allow_unlabeled=True, random_state=1
)


In [ ]:
Y = np.sum(Y, axis=1)

In [ ]:
print(X.shape)
clf = SVC(kernel="linear", decision_function_shape="ovr")
clf.fit(X, Y)
clf.classes_

In [ ]:
print(np.absolute(clf.coef_))

In [ ]:
import matplotlib.pyplot as plt
for i in range(clf.coef_.shape[0]):
    plt.plot(np.absolute(clf.coef_[i, :]), marker="o", linestyle="-")
    
plt.plot(np.nanmean(np.absolute(clf.coef_), axis=0), color="black")

In [ ]:
from sklearn.preprocessing import LabelEncoder
y = ["sLTP", "sLTP", "Stable", "sLTD", "Stable", "sLTD"]
le = LabelEncoder()
le.fit(y)

In [ ]:
coded = le.transform(y)
print(coded)

In [ ]:
a = "distance_coactivity_rate"
if "distance_coactivity_rate" in a or"distrbution" in a:
    print("Present")

In [ ]:
a = np.array([1,1,1,1,2,2,2,2,3,3,3,3]).reshape(3,4)
d = {"A": a, "B": a, "C": a}

In [ ]:
import pandas as pd
dfs = []
sub_count = 1
rm_vals = [1,2,3]
for key, value in d.items():
    for v in range(value.shape[1]):
        data=value[:, v]
        g = [key for x in range(len(data))]
        sub = [sub_count for x in range(len(data))]
        temp_dict = {"subject": sub, "data": data, "group": g, "rm_val": rm_vals}
        temp_df = pd.DataFrame(temp_dict)
        dfs.append(temp_df.dropna())
        sub_count = sub_count + 1
test_df = pd.concat(dfs)
test_df

In [ ]:
a = (1 < 0.05) *1
a

In [ ]:
a = list(np.array(list(range(10))).astype(str))

In [ ]:
a

In [ ]:
a = np.array([1,2,3,4])
d = {"a": a.reshape(-1,1), "b": a.reshape(-1,1)}

In [ ]:
c = list(d.values())
10 - np.nanmean(np.nan)

In [ ]:
position_bins = np.array([5, 10, 15, 20, 25, 30, 35, 40])
cluster_dist=10
DISTANT_BIN = 35

near_bin_idx = np.nonzero(position_bins == cluster_dist)[0][0]
dist_bin_idx = np.nonzero(position_bins >= DISTANT_BIN)[0][0]

In [ ]:
a = np.array([2,3,4,5,6,7])
b = np.array([1,2,3,4,5,6])

In [ ]:
c = np.vstack([a,b])
c

In [ ]:
np.nanmean(c, axis=1)

In [ ]:
a = ["A", "B", "C", "A", "A", "C"]
b = ["a", "a", "c", "b", "b", "b"]



In [ ]:
a_i = [i for i,x in enumerate(a) if x=="A"]
b_i = [i for i,x in enumerate(b) if x=="b"]

In [ ]:
a_i

In [ ]:
b_i

In [ ]:
np.intersect1d(a_i, b_i)

In [ ]:
a = np.array([0, 1,1,1, 0, 0])

In [ ]:
set(a)

In [ ]:
a = ["A", "A", "B", "C"]
c = np.array([0,0,0,1,1,2,2])
b = np.zeros(len(c)).astype(int)

b[np.where(c==1)] = 10
print(b)

In [ ]:
idxs = np.array([0,4,0,0,0,0,0])
print(idxs)

In [ ]:
np.vstack((b, idxs)).T

In [ ]:
import numpy as np

a = np.array([1,1,1,1])

l = [a, a, a*2]

np.vstack(l).T

In [ ]:
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

In [ ]:
A = np.random.rand(100, 20).T

B = np.random.rand(100, 20).T

In [ ]:
C = corr2_coeff(A, B)

In [ ]:
C.shape

In [ ]:
np.dot(A, B.T).shape

In [ ]:
target_vector = np.array([a,a,a])
print(target_vector)
candidate_vector = np.array([a, a*2, a*3])
print(candidate_vector)

In [ ]:
distance = np.linalg.norm(target_vector - candidate_vector, axis=1)

In [ ]:
distance

In [ ]:
1 / candidate_vector

In [ ]:
l = ["JW139", "JW139", "JW139", "JW167", "JW306", "JW139"]



In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(l)
housing_cat_1hot

In [ ]:
from collections import defaultdict
temp = defaultdict(lambda: len(temp))
res = [temp[ele] for ele in l]

res

In [ ]:
import pandas as pd
import numpy as np

d = dict( A = np.array([1,2]), B = np.array([1,2,3,4]) )
    
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items() ]))

In [ ]:
melted = pd.melt(df, value_vars=d.keys(), var_name="Group", value_name="Data")

In [ ]:
melted.dropna(axis=0, how="any")

In [ ]:
a = np.array(1)
b = np.array(2)

(b - a) / (b + a)

In [ ]:
for x, y in zip(a,b):
    print(x)
    print(y)